# Working with NOAA ISD data using Python, Pandas, and PyAthena

We use PyAthena for running the queries and Pandas for the results.

## Introduction to NOAA ISD Data

The NOAA Integrated Surface Database (ISD) is made up of global hourly observations compiled from a variety of sources.
The data are stored on S3 in the bucket `s3://noaa-isd-pds/` as fixed width text files. 

We will take advantage of Athena's ability to run queries over objects in S3 buckets, and use it for digesting the ISD data into manageable pieces. 

In [ ]:
import boto3
import uuid
from pyathena import connect
import pandas as pd
import matplotlib.pyplot as plt
from mpl_toolkits.basemap import Basemap

The `athena_data_bucket` variable is where we will store our query results automatically from Athena. This bucket will be created for you if it does not exist. If it does exist, you will need write access to the bucket.

In [ ]:
athena_data_bucket = "isd-demo-"
conn = connect(s3_staging_dir="s3://" + athena_data_bucket + '-' + str(uuid.uuid4()),
               region_name=boto3.session.Session().region_name)

## Plot time series Temperature & Dewpoint
### Query to get the air temperature and dewpoint temperature for the KOKC (Oklahoma City) station for the year.

We select only quality controlled data by checking if the quality of the observation is '5'. '5' represents 'Passed all quality control checks, data originate from an NCEI data source.' per the [ISD documentation](https://www.ncei.noaa.gov/data/global-hourly/doc/isd-format-document.pdf)

In [ ]:
df = pd.read_sql("SELECT obs_date, obs_time, air_temp, air_dewpoint_temp FROM noaaisd.isd where station_call = 'KOKC ' and air_temp_quality = '5' and air_dewpoint_temp_quality = '5';", conn)
df

### Get datetime object & convert C to F

The data stored in ISD as scaled integers, so we first unscale (divide by 10) then convert to Fahrenheit

In [ ]:
df['obs_date_time'] = pd.to_datetime(df.obs_date + df.obs_time, format='%Y%m%d%H%M')
df['air_temp'] = (df.air_temp / 10.0) * (9/5) + 32
df['air_dewpoint_temp'] = (df.air_dewpoint_temp / 10.0) * (9/5) + 32
df

### Plot the time series of temperature and dewpoint

In [ ]:
ax = plt.gca()
df.plot(figsize=(20,10), title='KOKC', kind='line', x='obs_date_time', y='air_dewpoint_temp', color='green', ax=ax)
df.plot(kind='line', x='obs_date_time', y='air_temp', color='red', ax=ax)
plt.xlabel('Date')
plt.ylabel('Temperature (F)')

## Plot locations of all stations in ISD

In [ ]:
df = pd.read_sql("SELECT DISTINCT obs_latitude, obs_longitude FROM noaaisd.isd;", conn)
df

rescale the latitude and longitude

In [ ]:
df['obs_latitude'] = df.obs_latitude / 1000.0
df['obs_longitude'] = df.obs_longitude / 1000.0
df

Plot a map, and plot the locations of the stations

In [ ]:
plt.subplots(figsize=(50,50))
m = Basemap(
    projection='merc', 
    llcrnrlat=-89.0, 
    urcrnrlat=89.0, 
    llcrnrlon=-180.0, 
    urcrnrlon=180.0,
    resolution='l'
)

m.drawmapboundary(fill_color='#ffffff')
m.drawcoastlines(color='#999999', linewidth=.4)

m.scatter(df.obs_longitude.tolist(), df.obs_latitude.tolist(), s=20, zorder=2, latlon=True)

plt.show()